In [2]:
import nltk
import re
from nltk.util import ngrams
from difflib import SequenceMatcher
from string import punctuation
from termcolor import colored
from fuzzysearch import find_near_matches

In [3]:
class Matcher: 
    def __init__(self, fileA, fileB, threshold, ngramSize):
        """
        Gets the texts from the files, tokenizes them, 
        cleans them up as necessary. 
        """
        self.threshold = threshold
        
        self.filenameA = fileA
        self.filenameB = fileB
        
        self.textA = self.readFile(fileA)
        self.textB = self.readFile(fileB)
        
        textATokens = self.tokenize(self.textA)
        textBTokens = self.tokenize(self.textB)
        
        self.textAgrams = list(ngrams(textATokens, ngramSize))
        self.textBgrams = list(ngrams(textBTokens, ngramSize))
        
    def readFile(self, filename): 
        """ Reads the file in memory. """
        return open(filename).read()

    def tokenize(self, text): 
        """ Tokenizes the text, breaking it up into words. """
        return nltk.word_tokenize(text.lower())

    def gramsToString(self, grams): 
        """
        Takes a list of tuples (3-grams, 4-grams, etc.) 
        and stitches it back together into a string, so that
        we can search the non-tokenized text for the string later. 
        """
        string = " ".join(grams[0][:-1])
        for gram in grams:
            lastGram = gram[-1]
            if lastGram not in punctuation: 
                string += " " + lastGram
            else: 
                string += lastGram
        return string
    
    def getMatch(self, match, textA, textB): 
        """ 
        Takes the match object returned by get_matching_blocks() and
        gets the matched n-gram. It uses gramsToString() to 
        reformat this into a string.
        """
        textAs, textBs = [], []
        for i in range(match.size):
            textAs.append(textA[match.a+i])
            textBs.append(textB[match.b+i])
        return (self.gramsToString(textAs), self.gramsToString(textBs))

    def match(self): 
        """
        This does the main work of finding matching n-gram sequences between
        the texts.
        """
        sequence = SequenceMatcher(None,self.textAgrams,self.textBgrams)
        matchingBlocks = sequence.get_matching_blocks()

        # Only return the matching sequences that are higher than the 
        # threshold given by the user. 
        highMatchingBlocks = [match for match in matchingBlocks if match.size > self.threshold]

        for match in highMatchingBlocks: 
            out = self.getMatch(match, self.textAgrams, self.textBgrams)
            print('\n', out)
            self.findInText(out[0], self.textA, self.filenameA, 20)
            self.findInText(out[1], self.textB, self.filenameB, 20)
        
    def findInText(self, needle, haystack, haystackName, context):
        """
        This takes the matches found by match() and tries to find that match
        again in the text, so that we can return some context. Uses the
        fuzzysearch library, because I couldn't find anything better.
        """
        m = find_near_matches(needle, haystack, max_l_dist=2)
        
        if len(m) > 0: 
            m = m[0] # just get the first match for now. TODO: get all of them
            
            before = haystack[m.start-context:m.start]
            match  = colored(haystack[m.start:m.end], 'red')
            after  = haystack[m.end:m.end+context]    
            
            contextualized = before + match + after
            cleaned = re.sub( '\s+', ' ', contextualized ).strip()
            print(colored(haystackName, 'green') + ": " + cleaned)
        else: 
            print('Couldn\'t find this match in file: ', haystackName)
        

In [5]:
myMatch = Matcher('texts/milton.txt', 'texts/kjv.txt', 2, 3)
myMatch.match()


 (', and thou shalt be', ', and thou shalt be')
texts/milton.txt: e of streaming light, And thou shalt be our star of Arcady,
texts/kjv.txt: hall bruise thy head, and thou shalt bruise his heel. 3:16

 (', and herds, and', ', and herds, and')
texts/milton.txt: eerfull haunt of men, and herds, And sits as safe as in
texts/kjv.txt: th Abram, had flocks, and herds, and tents. 13:6 And th

 (', as he said,', ', as he said,')
texts/milton.txt: in another Countrey, as he said, Bore a bright golde
texts/kjv.txt: one giving him drink, she said, I will draw water f

 ('out of his hand, and', 'out of his hand, and')
texts/milton.txt: wn, wrest his Glass out of his hand, and break it against th
texts/kjv.txt: he cast the tables out of his hands, and brake them beneath

 (', if it seem good to', ', if it seem good to')
texts/milton.txt: amp: Imprimatur, If it seem good to the reverend Master
texts/kjv.txt: vineyard than it; or, if it seem good to thee, I will give t

 (', that the word of', ', th

In [6]:
myMatch = Matcher('texts/portrait.txt', 'texts/kjv.txt', 3, 4)

In [7]:
myMatch.match()


 (', we beseech thee, o lord,', ', we beseech thee, o lord,')
Couldn't find this match in file:  texts/portrait.txt
Couldn't find this match in file:  texts/kjv.txt

 ('better for him that a millstone were', 'better for him that a millstone were')
Couldn't find this match in file:  texts/portrait.txt
texts/kjv.txt: ieve in me, it were better for him that a millstone were hanged about his ne

 ('depart from me, ye cursed, into everlasting fire', 'depart from me, ye cursed, into everlasting fire')
texts/portrait.txt: it lay, he said: --Depart from me, ye cursed, into everlasting fire! Taking Stephen's
texts/kjv.txt: m on the left hand, Depart from me, ye cursed, into everlasting fire, prepared for the d

 ('prepared for the devil and his angels', 'prepared for the devil and his angels')
Couldn't find this match in file:  texts/portrait.txt
texts/kjv.txt: o everlasting fire, prepared for the devil and his angels: 25:42 For I was an

 ('o grave , where is thy victory?', 'o grave , where i

In [9]:
myMatch = Matcher('texts/yeats.txt', 'texts/kjv.txt', 3, 4)
myMatch.match()


 ('eat , drink, and be merry.', 'eat , drink, and be merry.')
texts/yeats.txt: eal with us Shall eat, drink, and be merry. SHEMUS (_to_ MAR
texts/kjv.txt: s; take thine ease, eat, drink, and be merry. 12:20 But God said
